### glacier elevation change tiles processing
1. mosaic of images;   
2. subseting of image to hma region.


In [5]:
import os
import rasterio as rio
import geopandas as gpd
from glob import glob
import matplotlib.pyplot as plt
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling


### Mosaic

['data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/dhdt/N36E099_2000-01-01_2020-01-01_dhdt.tif',
 'data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/dhdt/N31E077_2000-01-01_2020-01-01_dhdt.tif',
 'data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/dhdt/N34E074_2000-01-01_2020-01-01_dhdt.tif',
 'data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/dhdt/N31E079_2000-01-01_2020-01-01_dhdt.tif',
 'data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/dhdt/N30E081_2000-01-01_2020-01-01_dhdt.tif',
 'data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/dhdt/N32E077_2000-01-01_2020-01-01_dhdt.tif',
 'data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/dhdt/N41E077_2000-01-01_2020-01-01_dhdt.tif',
 'data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/dhdt/N37E087_2000-01-01_2020-01-01_dhdt.tif',
 'data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/dhdt/N37E091_2000-01-01_2020-01-01_dhdt.tif',
 'data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/dhdt/N30E094_2000-01-01_2020-01-01_d

In [12]:
paths_tile_ls = glob('data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/dhdt/*.tif')
path_mosaic = 'data/glacier-height/hugonent-gla-dset/gla-change_map_2000_2019/dhdt_mosaic.tif'

srcs_to_mosaic = []
# Use the CRS of the first raster as the target CRS
with rio.open(paths_tile_ls[0]) as ref_src:
    target_crs = ref_src.crs

for fp in paths_tile_ls:
    src = rio.open(fp)
    if src.crs != target_crs:
        # Reproject to target CRS
        data = src.read(
            out_shape=(
                src.count,
                int(src.height),
                int(src.width)
            ),
            resampling=Resampling.nearest
        )
        transform, width, height = rio.warp.calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        memfile = rio.io.MemoryFile()
        with memfile.open(**kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rio.band(src, i),
                    destination=rio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest
                )
            srcs_to_mosaic.append(memfile.open())
    else:
        srcs_to_mosaic.append(src)

mosaic_arr, mosaic_trans = merge(srcs_to_mosaic)
mosaic_meta = srcs_to_mosaic[0].meta.copy()
mosaic_meta.update({
    "height": mosaic_arr.shape[1],
    "width": mosaic_arr.shape[2],
    "transform": mosaic_trans
    })


In [25]:
from rasterio.warp import calculate_default_transform

def raster_reproj(src, target_crs="EPSG:4326", resample_method=Resampling.nearest):
    """
    Reproject a raster to a specified CRS.
    Parameters:
        src (rasterio.io.DatasetReader): Source raster.
        target_crs (str): Target CRS in EPSG format.
        resample_method (Resampling): Resampling method to use.        
    Returns:
        src_reproj (numpy.ndarray): Reprojected raster data.
    """
    transform, width, height = calculate_default_transform(
                src.crs, target_crs, src.width, src.height, *src.bounds)    
    kwargs = src.profile.copy()
    kwargs.update({
        'crs': target_crs, 'transform': transform,
        'width': width, 'height': height})
    
    memfile = rio.io.MemoryFile()
    with memfile.open(**kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rio.band(src, i),
                destination=rio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=target_crs,
                resampling=resample_method
            )
        src_reproj = memfile.open()  # Open the memory file as a rasterio dataset
    return src_reproj

src = rio.open(fp)
dst_crs = "EPSG:4326"
src_reproj = raster_reproj(src, dst_crs)
src_reproj.read()


array([[[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.]]],
      shape=(1, 344, 880), dtype=float32)

In [40]:
paths_tile_ls = glob('data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/dhdt/*.tif')
path_mosaic = 'data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/dhdt/dhdt_mosaic.tif'

srcs_to_mosaic = []
for fp in paths_tile_ls:
    src = rio.open(fp)
    src_reproj2wgs84 = raster_reproj(src, target_crs="EPSG:4326")
    srcs_to_mosaic.append(src_reproj2wgs84)

mosaic_arr, mosaic_trans = merge(srcs_to_mosaic)
mosaic_meta = src.meta.copy()
mosaic_meta.update({
    "height": mosaic_arr.shape[1],
    "width": mosaic_arr.shape[2],
    "transform": mosaic_trans,
    "crs": "EPSG:4326"  # Ensure the CRS is set to WGS84
    })
# Write the mosaic raster to disk
with rio.open(path_mosaic, 'w', **mosaic_meta) as dest:
    dest.write(mosaic_arr)



In [45]:
# paths_tile_ls = glob('data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/dhdt/*.tif')
# path_mosaic = 'data/glacier-height/hugonent-gla-dset/gla-change_map_2000_2019/dhdt_mosaic.tif'

# srcs_to_mosaic = []
# for fp in paths_tile_ls:
#     src = rio.open(fp)
#     srcs_to_mosaic.append(src)
# mosaic_arr, mosaic_trans = merge(srcs_to_mosaic)
# mosaic_meta = src.meta.copy()
# mosaic_meta.update({
#     "height": mosaic_arr.shape[1],
#     "width": mosaic_arr.shape[2],
#     "transform": mosaic_trans
#     })

# # # Write the mosaic raster to disk
# # with rio.open(path_mosaic, 'w', **mosaic_meta) as dest:
# #     dest.write(mosaic_arr)



